Modified from https://github.com/melanieabrams-pub/RH-seq_with_barcoding/blob/main/annotate_poolfile_for_barseq_v1.2_JSkerker.ipynb and 
https://github.com/melanieabrams-pub/kluyv_RH-seq_with_barcoding/blob/main/annotate_poolfile_kluyv.py

In [1]:
import pandas as pd
import os
import matplotlib.pyplot as plt

#makes viewing pandas tables better
pd.set_option('display.max_colwidth', 0)

### Load GFF annotations

In [2]:
#you don't need to change the path below - this is the most recent annotation file (5/12/22)
gff_file = '/usr2/people/shollyt22/shollyt22/TnSeq_BarSeq_sequencings/Barseq_July_2023/OORB003_TnSeq/for_analysis/modified_gff_file_finalized_08_08_2023'

In [5]:
import pandas as pd

def count_unique_genes_gff(gff_file):
    # Read the GFF file
    gff_data = pd.read_csv(gff_file, sep='\t', comment='#')
    
    # Check if 'ID' column is present
    if 'ID' in gff_data.columns:
        # Extract the unique gene IDs from the 'ID' column
        unique_genes = gff_data['ID'].unique()
        
        # Filter out any NaN values
        unique_genes = unique_genes[~pd.isnull(unique_genes)]
        
        return len(unique_genes)
    else:
        return 0

gff_file_path = '/usr2/people/shollyt22/shollyt22/TnSeq_BarSeq_sequencings/Barseq_July_2023/OORB003_TnSeq/for_analysis/modified_gff_file_finalized_08_08_2023'
total_genes = count_unique_genes_gff(gff_file_path)
print(f'Total number of genes in GFF file: {total_genes}')


Total number of genes in GFF file: 9110


In [6]:
gff = pd.read_csv(gff_file, sep='\t')

In [7]:
t = gff['scaffold'].unique()
print(t)

['chr1' 'chr2' 'chr3' 'chr4' 'chr5' 'chr6' 'chr7']


In [8]:
gff.head()

,scaffold,source,type,start,stop,strand,info,ID
0,chr1,JGI,CDS,1381.0,2142.0,+,ID=gene_1,proteinId=2114025
1,chr1,JGI,CDS,2744.0,3343.0,-,ID=gene_2,proteinId=2293935
2,chr1,JGI,CDS,3344.0,3817.0,-,ID=gene_3,proteinId=2293936
3,chr1,JGI,CDS,5330.0,6385.0,-,ID=gene_4,proteinId=2051335
4,chr1,JGI,CDS,15458.0,25533.0,+,ID=gene_5,proteinId=2121898


In [9]:
gff

,scaffold,source,type,start,stop,strand,info,ID
0,chr1,JGI,CDS,1381.0,2142.0,+,ID=gene_1,proteinId=2114025
1,chr1,JGI,CDS,2744.0,3343.0,-,ID=gene_2,proteinId=2293935
2,chr1,JGI,CDS,3344.0,3817.0,-,ID=gene_3,proteinId=2293936
3,chr1,JGI,CDS,5330.0,6385.0,-,ID=gene_4,proteinId=2051335
4,chr1,JGI,CDS,15458.0,25533.0,+,ID=gene_5,proteinId=2121898
...,...,...,...,...,...,...,...,...
9105,chr7,JGI,CDS,4089438.0,4091337.0,+,ID=gene_10836,proteinId=103797
9106,chr7,JGI,CDS,4095568.0,4097284.0,+,ID=gene_10837,proteinId=2071098
9107,chr7,JGI,CDS,4098122.0,4098907.0,-,ID=gene_10838,proteinId=2114022
9108,chr7,JGI,CDS,4100606.0,4101293.0,-,ID=gene_10840,proteinId=2114023


### Match barcode insert locations to gene annotations

In [10]:
!pwd

/auto/sahara/namib/home/shollyt22/shollyt22/TnSeq_BarSeq_sequencings/Sequencing_with_the_13k_mutants


In [11]:
#CHANGE IF NEEDED: load COMBINED poolfile generated in TNseq mapping step

poolfile = '/usr2/people/shollyt22/shollyt22/TnSeq_BarSeq_sequencings/Sequencing_with_the_13k_mutants/for_analysis/all_fastqs_available/TNSeq_13k_mutants_mapping_output_all_fastqs_available/Tnseq_13k_mutants_all_digestion_time_poolfile'

#CHANGE IF NEEDED: annotated pool file path to output

annotated_poolfile_path = '/usr2/people/shollyt22/shollyt22/TnSeq_BarSeq_sequencings/Sequencing_with_the_13k_mutants/for_analysis/all_fastqs_available/TNSeq_13k_mutants_mapping_output_all_fastqs_available/Tnseq_13k_mutants_all_digestion_time_poolfile_annotated.csv'

In [12]:
df = pd.read_csv(poolfile,sep='\t')
df.head()

,barcode,rcbarcode,nTot,n,scaffold,strand,pos,type,nMainLocation,nInsert,All genomic mappings,All insert mappings
0,TTCATTCAGGATCTTTGCTC,GAGCAAAGATCCTGAATGAA,601,601,chr1,+,8764372,Single,563,14,"[('chr1:8764372:+', 563), ('chr5:3325007:-', 3), ('chr7:2406660:-', 2), ('chr6:477092:-', 1), ('chr7:1928216:+', 1), ('chr4:2631691:-', 1), ('chr3:313312:+', 1), ('chr6:733284:-', 1), ('chr1:4277183:-', 1), ('chr1:1979828:+', 1), ('chr2:991296:-', 1), ('chr3:838055:-', 1), ('chr3:3091523:+', 1), ('chr1:7763084:+', 1), ('chr6:3244298:+', 1), ('chr1:6914244:-', 1), ('chr3:2416675:+', 1), ('chr1:9602398:+', 1), ('chr1:10585823:+', 1), ('chr1:8696906:+', 1), ('chr4:2000475:-', 1), ('chr7:1733638:-', 1)]","{'Null:0:+': 0, 'insert:1138:-': 1, 'insert:19:+': 1, 'insert:1:+': 4, 'insert:4:+': 1, 'insert:130:+': 3, 'insert:1484:+': 1, 'insert:459:+': 1, 'insert:13:+': 1, 'insert:869:+': 1}"
1,ACGTATTCGCAGTCCTCATA,TATGAGGACTGCGAATACGT,52,52,chr1,-,526973,Single,50,0,"[('chr1:526973:-', 50), ('chr1:972680:+', 1), ('chr6:932721:-', 1)]",{'Null:0:+': 0}
2,GCGGCGGCAGCGTTCCTCGC,GCGAGGAACGCTGCCGCCGC,498,498,chr1,-,2235514,Single,450,14,"[('chr1:2235514:-', 450), ('chr7:2406660:-', 2), ('chr7:159854:-', 2), ('chr2:2382780:-', 1), ('chr3:2906544:-', 1), ('chr2:2797042:-', 1), ('chr3:1386264:-', 1), ('chr1:9117601:+', 1), ('chr1:2235513:-', 1), ('chr2:1023360:+', 1), ('chr5:3144776:-', 1), ('chr5:3773802:+', 1), ('chr5:3199747:-', 1), ('chr7:3111985:+', 1), ('chr1:4277183:-', 1), ('chr1:6419382:-', 1), ('chr3:1018484:+', 1), ('chr2:1802697:+', 1), ('chr1:8693211:+', 1), ('chr3:409126:+', 1), ('chr1:9400362:+', 1), ('chr5:3325007:-', 1), ('chr1:3432198:+', 1), ('chr2:2283320:+', 1), ('chr4:2420774:+', 1), ('chr6:3402852:+', 1), ('chr3:2949454:+', 1), ('chr7:2350573:+', 1), ('chr4:2678882:+', 1), ('chr2:4643869:-', 1), ('chr7:3369397:+', 1), ('chr7:3502691:-', 1), ('chr1:9554108:-', 1)]","{'Null:0:+': 0, 'insert:365:+': 1, 'insert:1:+': 5, 'insert:61:-': 1, 'plasmid:1154:-': 1, 'insert:130:+': 1, 'insert:1470:+': 1, 'insert:5:+': 1, 'insert:1783:-': 1, 'insert:4:+': 1, 'insert:18:+': 1}"
3,TTCGCGTGGGTGAATTAACT,AGTTAATTCACCCACGCGAA,1806,1806,chr5,+,2435148,Single,1660,43,"[('chr5:2435148:+', 1660), ('chr7:2406660:-', 10), ('chr3:2141768:+', 3), ('chr5:3325007:-', 3), ('chr1:9415816:+', 3), ('chr5:2435149:+', 3), ('chr5:2435205:-', 2), ('chr1:9554108:-', 2), ('chr3:2949780:+', 2), ('chr6:201290:+', 2), ('chr1:6914244:-', 2), ('chr5:2633031:-', 2), ('chr1:7572780:-', 1), ('chr2:1803458:-', 1), ('chr5:2574087:-', 1), ('chr7:2035049:-', 1), ('chr7:2775803:+', 1), ('chr3:2410718:-', 1), ('chr3:2618659:-', 1), ('chr1:6943496:-', 1), ('chr2:4994449:+', 1), ('chr2:4661375:+', 1), ('chr7:1074090:+', 1), ('chr7:1049234:+', 1), ('chr6:3125324:-', 1), ('chr2:4512077:+', 1), ('chr2:1751841:-', 1), ('chr1:6903684:-', 1), ('chr1:391550:-', 1), ('chr2:3965159:+', 1), ('chr7:1885715:-', 1), ('chr2:2382780:-', 1), ('chr5:464496:-', 1), ('chr2:3730704:+', 1), ('chr1:6024539:+', 1), ('chr1:6347373:+', 1), ('chr1:6012377:-', 1), ('chr1:9969230:-', 1), ('chr3:142030:+', 1), ('chr1:1505216:+', 1), ('chr2:5223225:-', 1), ('chr2:5364748:+', 1), ('chr1:7624221:-', 1), ('chr5:2435151:+', 1), ('chr6:1178030:-', 1), ('chr2:4314401:+', 1), ('chr6:1653165:-', 1), ('chr1:3843797:+', 1), ('chr5:2435190:-', 1), ('chr3:2931743:-', 1), ('chr2:2044521:+', 1), ('chr4:2071158:-', 1), ('chr2:1751846:+', 1), ('chr2:4994013:-', 1), ('chr1:6383044:-', 1), ('chr1:2396701:+', 1), ('chr3:3753786:-', 1), ('chr7:2321031:+', 1), ('chr4:127963:+', 1), ('chr4:2045081:+', 1), ('chr2:1589959:-', 1), ('chr2:3539420:+', 1), ('chr7:2206210:+', 1), ('chr6:2450270:-', 1), ('chr1:7383411:-', 1), ('chr7:2484936:+', 1), ('chr1:1517359:-', 1), ('chr7:3111985:+', 1), ('chr7:2626073:-', 1), ('chr1:821751:-', 1), ('chr7:601923:+', 1), ('chr3:3135700:-', 1), ('chr5:3772661:+', 1), ('chr5:3773803:+', 1), ('chr1:9316561:-', 1), ('chr1:3703136:-', 1), ('chr3:4590094:-', 1), ('chr6:2851498:+', 1), ('chr2:1807179:-', 1), ('ch

Check that each barcode in the dataframe is unique (the two numbers below should be the same):

In [13]:
print('Number of insertions in poolfile: {}'.format(df.shape[0]))
print('Number of unique barcodes in poolfile: {}'.format(len(df['barcode'].unique())))

Number of insertions in poolfile: 13364
Number of unique barcodes in poolfile: 13364


In [14]:
#df = df.set_index('transposonLocation') #This is what was there and it is showing error. The transposonlocation was changed to barcode in the next line
df = df.set_index('barcode')

In [15]:
df.columns

Index(['rcbarcode', 'nTot', 'n', 'scaffold', 'strand', 'pos', 'type',
       'nMainLocation', 'nInsert', 'All genomic mappings',
       'All insert mappings'],
      dtype='object')

In [16]:
df.head()

,rcbarcode,nTot,n,scaffold,strand,pos,type,nMainLocation,nInsert,All genomic mappings,All insert mappings
barcode,,,,,,,,,,,
TTCATTCAGGATCTTTGCTC,GAGCAAAGATCCTGAATGAA,601,601,chr1,+,8764372,Single,563,14,"[('chr1:8764372:+', 563), ('chr5:3325007:-', 3), ('chr7:2406660:-', 2), ('chr6:477092:-', 1), ('chr7:1928216:+', 1), ('chr4:2631691:-', 1), ('chr3:313312:+', 1), ('chr6:733284:-', 1), ('chr1:4277183:-', 1), ('chr1:1979828:+', 1), ('chr2:991296:-', 1), ('chr3:838055:-', 1), ('chr3:3091523:+', 1), ('chr1:7763084:+', 1), ('chr6:3244298:+', 1), ('chr1:6914244:-', 1), ('chr3:2416675:+', 1), ('chr1:9602398:+', 1), ('chr1:10585823:+', 1), ('chr1:8696906:+', 1), ('chr4:2000475:-', 1), ('chr7:1733638:-', 1)]","{'Null:0:+': 0, 'insert:1138:-': 1, 'insert:19:+': 1, 'insert:1:+': 4, 'insert:4:+': 1, 'insert:130:+': 3, 'insert:1484:+': 1, 'insert:459:+': 1, 'insert:13:+': 1, 'insert:869:+': 1}"
ACGTATTCGCAGTCCTCATA,TATGAGGACTGCGAATACGT,52,52,chr1,-,526973,Single,50,0,"[('chr1:526973:-', 50), ('chr1:972680:+', 1), ('chr6:932721:-', 1)]",{'Null:0:+': 0}
GCGGCGGCAGCGTTCCTCGC,GCGAGGAACGCTGCCGCCGC,498,498,chr1,-,2235514,Single,450,14,"[('chr1:2235514:-', 450), ('chr7:2406660:-', 2), ('chr7:159854:-', 2), ('chr2:2382780:-', 1), ('chr3:2906544:-', 1), ('chr2:2797042:-', 1), ('chr3:1386264:-', 1), ('chr1:9117601:+', 1), ('chr1:2235513:-', 1), ('chr2:1023360:+', 1), ('chr5:3144776:-', 1), ('chr5:3773802:+', 1), ('chr5:3199747:-', 1), ('chr7:3111985:+', 1), ('chr1:4277183:-', 1), ('chr1:6419382:-', 1), ('chr3:1018484:+', 1), ('chr2:1802697:+', 1), ('chr1:8693211:+', 1), ('chr3:409126:+', 1), ('chr1:9400362:+', 1), ('chr5:3325007:-', 1), ('chr1:3432198:+', 1), ('chr2:2283320:+', 1), ('chr4:2420774:+', 1), ('chr6:3402852:+', 1), ('chr3:2949454:+', 1), ('chr7:2350573:+', 1), ('chr4:2678882:+', 1), ('chr2:4643869:-', 1), ('chr7:3369397:+', 1), ('chr7:3502691:-', 1), ('chr1:9554108:-', 1)]","{'Null:0:+': 0, 'insert:365:+': 1, 'insert:1:+': 5, 'insert:61:-': 1, 'plasmid:1154:-': 1, 'insert:130:+': 1, 'insert:1470:+': 1, 'insert:5:+': 1, 'insert:1783:-': 1, 'insert:4:+': 1, 'insert:18:+': 1}"
TTCGCGTGGGTGAATTAACT,AGTTAATTCACCCACGCGAA,1806,1806,chr5,+,2435148,Single,1660,43,"[('chr5:2435148:+', 1660), ('chr7:2406660:-', 10), ('chr3:2141768:+', 3), ('chr5:3325007:-', 3), ('chr1:9415816:+', 3), ('chr5:2435149:+', 3), ('chr5:2435205:-', 2), ('chr1:9554108:-', 2), ('chr3:2949780:+', 2), ('chr6:201290:+', 2), ('chr1:6914244:-', 2), ('chr5:2633031:-', 2), ('chr1:7572780:-', 1), ('chr2:1803458:-', 1), ('chr5:2574087:-', 1), ('chr7:2035049:-', 1), ('chr7:2775803:+', 1), ('chr3:2410718:-', 1), ('chr3:2618659:-', 1), ('chr1:6943496:-', 1), ('chr2:4994449:+', 1), ('chr2:4661375:+', 1), ('chr7:1074090:+', 1), ('chr7:1049234:+', 1), ('chr6:3125324:-', 1), ('chr2:4512077:+', 1), ('chr2:1751841:-', 1), ('chr1:6903684:-', 1), ('chr1:391550:-', 1), ('chr2:3965159:+', 1), ('chr7:1885715:-', 1), ('chr2:2382780:-', 1), ('chr5:464496:-', 1), ('chr2:3730704:+', 1), ('chr1:6024539:+', 1), ('chr1:6347373:+', 1), ('chr1:6012377:-', 1), ('chr1:9969230:-', 1), ('chr3:142030:+', 1), ('chr1:1505216:+', 1), ('chr2:5223225:-', 1), ('chr2:5364748:+', 1), ('chr1:7624221:-', 1), ('chr5:2435151:+', 1), ('chr6:1178030:-', 1), ('chr2:4314401:+', 1), ('chr6:1653165:-', 1), ('chr1:3843797:+', 1), ('chr5:2435190:-', 1), ('chr3:2931743:-', 1), ('chr2:2044521:+', 1), ('chr4:2071158:-', 1), ('chr2:1751846:+', 1), ('chr2:4994013:-', 1), ('chr1:6383044:-', 1), ('chr1:2396701:+', 1), ('chr3:3753786:-', 1), ('chr7:2321031:+', 1), ('chr4:127963:+', 1), ('chr4:2045081:+', 1), ('chr2:1589959:-', 1), ('chr2:3539420:+', 1), ('chr7:2206210:+', 1), ('chr6:2450270:-', 1), ('chr1:7383411:-', 1), ('chr7:2484936:+', 1), ('chr1:1517359:-', 1), ('chr7:3111985:+', 1), ('chr7:2626073:-', 1), ('chr1:821751:-', 1), ('chr7:601923:+', 1), ('chr3:3135700:-', 1), ('chr5:3772661:+', 1), ('chr5:3773803:+', 1), ('chr1:9316561:-', 1), ('chr1:3703136:-', 1), ('chr3:4590094:-', 1), ('chr6:2851498:+', 1), ('chr2:1807179:-', 1), (

In [17]:
df=df.drop(df.columns[-1],axis=1)

In [18]:
# df.columns = ['rcbarcode','nTot', 'n', 'scaffold', 'strand', 'pos', 'type',
#        'nMainLocation', 'nInsert', 'All genomic mappings',
#        'All insert mappings']

In [19]:
df.head()

,rcbarcode,nTot,n,scaffold,strand,pos,type,nMainLocation,nInsert,All genomic mappings
barcode,,,,,,,,,,
TTCATTCAGGATCTTTGCTC,GAGCAAAGATCCTGAATGAA,601,601,chr1,+,8764372,Single,563,14,"[('chr1:8764372:+', 563), ('chr5:3325007:-', 3), ('chr7:2406660:-', 2), ('chr6:477092:-', 1), ('chr7:1928216:+', 1), ('chr4:2631691:-', 1), ('chr3:313312:+', 1), ('chr6:733284:-', 1), ('chr1:4277183:-', 1), ('chr1:1979828:+', 1), ('chr2:991296:-', 1), ('chr3:838055:-', 1), ('chr3:3091523:+', 1), ('chr1:7763084:+', 1), ('chr6:3244298:+', 1), ('chr1:6914244:-', 1), ('chr3:2416675:+', 1), ('chr1:9602398:+', 1), ('chr1:10585823:+', 1), ('chr1:8696906:+', 1), ('chr4:2000475:-', 1), ('chr7:1733638:-', 1)]"
ACGTATTCGCAGTCCTCATA,TATGAGGACTGCGAATACGT,52,52,chr1,-,526973,Single,50,0,"[('chr1:526973:-', 50), ('chr1:972680:+', 1), ('chr6:932721:-', 1)]"
GCGGCGGCAGCGTTCCTCGC,GCGAGGAACGCTGCCGCCGC,498,498,chr1,-,2235514,Single,450,14,"[('chr1:2235514:-', 450), ('chr7:2406660:-', 2), ('chr7:159854:-', 2), ('chr2:2382780:-', 1), ('chr3:2906544:-', 1), ('chr2:2797042:-', 1), ('chr3:1386264:-', 1), ('chr1:9117601:+', 1), ('chr1:2235513:-', 1), ('chr2:1023360:+', 1), ('chr5:3144776:-', 1), ('chr5:3773802:+', 1), ('chr5:3199747:-', 1), ('chr7:3111985:+', 1), ('chr1:4277183:-', 1), ('chr1:6419382:-', 1), ('chr3:1018484:+', 1), ('chr2:1802697:+', 1), ('chr1:8693211:+', 1), ('chr3:409126:+', 1), ('chr1:9400362:+', 1), ('chr5:3325007:-', 1), ('chr1:3432198:+', 1), ('chr2:2283320:+', 1), ('chr4:2420774:+', 1), ('chr6:3402852:+', 1), ('chr3:2949454:+', 1), ('chr7:2350573:+', 1), ('chr4:2678882:+', 1), ('chr2:4643869:-', 1), ('chr7:3369397:+', 1), ('chr7:3502691:-', 1), ('chr1:9554108:-', 1)]"
TTCGCGTGGGTGAATTAACT,AGTTAATTCACCCACGCGAA,1806,1806,chr5,+,2435148,Single,1660,43,"[('chr5:2435148:+', 1660), ('chr7:2406660:-', 10), ('chr3:2141768:+', 3), ('chr5:3325007:-', 3), ('chr1:9415816:+', 3), ('chr5:2435149:+', 3), ('chr5:2435205:-', 2), ('chr1:9554108:-', 2), ('chr3:2949780:+', 2), ('chr6:201290:+', 2), ('chr1:6914244:-', 2), ('chr5:2633031:-', 2), ('chr1:7572780:-', 1), ('chr2:1803458:-', 1), ('chr5:2574087:-', 1), ('chr7:2035049:-', 1), ('chr7:2775803:+', 1), ('chr3:2410718:-', 1), ('chr3:2618659:-', 1), ('chr1:6943496:-', 1), ('chr2:4994449:+', 1), ('chr2:4661375:+', 1), ('chr7:1074090:+', 1), ('chr7:1049234:+', 1), ('chr6:3125324:-', 1), ('chr2:4512077:+', 1), ('chr2:1751841:-', 1), ('chr1:6903684:-', 1), ('chr1:391550:-', 1), ('chr2:3965159:+', 1), ('chr7:1885715:-', 1), ('chr2:2382780:-', 1), ('chr5:464496:-', 1), ('chr2:3730704:+', 1), ('chr1:6024539:+', 1), ('chr1:6347373:+', 1), ('chr1:6012377:-', 1), ('chr1:9969230:-', 1), ('chr3:142030:+', 1), ('chr1:1505216:+', 1), ('chr2:5223225:-', 1), ('chr2:5364748:+', 1), ('chr1:7624221:-', 1), ('chr5:2435151:+', 1), ('chr6:1178030:-', 1), ('chr2:4314401:+', 1), ('chr6:1653165:-', 1), ('chr1:3843797:+', 1), ('chr5:2435190:-', 1), ('chr3:2931743:-', 1), ('chr2:2044521:+', 1), ('chr4:2071158:-', 1), ('chr2:1751846:+', 1), ('chr2:4994013:-', 1), ('chr1:6383044:-', 1), ('chr1:2396701:+', 1), ('chr3:3753786:-', 1), ('chr7:2321031:+', 1), ('chr4:127963:+', 1), ('chr4:2045081:+', 1), ('chr2:1589959:-', 1), ('chr2:3539420:+', 1), ('chr7:2206210:+', 1), ('chr6:2450270:-', 1), ('chr1:7383411:-', 1), ('chr7:2484936:+', 1), ('chr1:1517359:-', 1), ('chr7:3111985:+', 1), ('chr7:2626073:-', 1), ('chr1:821751:-', 1), ('chr7:601923:+', 1), ('chr3:3135700:-', 1), ('chr5:3772661:+', 1), ('chr5:3773803:+', 1), ('chr1:9316561:-', 1), ('chr1:3703136:-', 1), ('chr3:4590094:-', 1), ('chr6:2851498:+', 1), ('chr2:1807179:-', 1), ('chr7:1926954:-', 1), ('chr2:4267615:+', 1)]"
TTCGTTTAGTTGTCTAAGTA,TACTTAGACAACTAAACGAA,401,401,chr3,-,1481765,Concat,318,50,"[('chr3:1481765:-', 318), ('chr7:2406660:-', 2), ('chr1:2541261:+', 2), ('chr1:6517427:-', 1), ('chr1:8477319:-', 1), ('chr7:2592385:-', 1), ('chr5:2283227:-', 1), ('chr3:3582433:+', 1), ('chr2:3041586:-', 1), ('chr1:9415816:+', 1), ('chr1:9120459:+', 1), ('chr2:1702333:-', 1), ('chr3:4590094:-', 1)

In [21]:
#this takes a few minutes
def get_gene_by_bc_position(pos):
    
    gene_list = [scaffold_gff.loc[interval]['ID'] for interval in idx if pos in interval]
    
    #barcode position not in any genes
    if not gene_list:
        return ''
    #barcode position is in one gene
    if len(gene_list) == 1:
        return gene_list[0]
    #barcode position is in multiple genes (should be rare)
    else:
        return ';'.join(gene_list)

barcode_to_gene_dict = {}

for scaffold in sorted(gff['scaffold'].unique()):
    
    scaffold_gff = gff[gff['scaffold']==scaffold]
    scaffold_barcodes = df[df['scaffold']==scaffold]
    
    #divide scaffold_gff.shape by 2 here to get number of genes (original shape includes promoters)
    print('Mapping {} barcodes on {} to {} genes on {}'.format(scaffold_barcodes.shape[0], scaffold,
                                                              round(scaffold_gff.shape[0]/2), scaffold))

    #turn the start and stop of each gene into an interval
    v = scaffold_gff.loc[:, 'start':'stop'].apply(tuple, 1).tolist()
    idx = pd.IntervalIndex.from_tuples(v, closed='both') 
    scaffold_gff = scaffold_gff.set_index(idx)
    
    #for each barcode, check if it's in each start/stop interval
    barcode_to_gene_dict.update(scaffold_barcodes['pos'].apply(get_gene_by_bc_position).to_dict())
    
df['gene'] = df.index.map(barcode_to_gene_dict)

Mapping 4186 barcodes on chr1 to 1356 genes on chr1
Mapping 2234 barcodes on chr2 to 715 genes on chr2
Mapping 1732 barcodes on chr3 to 605 genes on chr3
Mapping 1266 barcodes on chr4 to 535 genes on chr4
Mapping 1040 barcodes on chr5 to 492 genes on chr5
Mapping 1016 barcodes on chr6 to 396 genes on chr6
Mapping 1890 barcodes on chr7 to 455 genes on chr7


In [23]:
import pandas as pd

def get_gene_by_bc_position(pos):
    gene_list = [scaffold_gff.loc[interval]['ID'] for interval in idx if pos in interval]
    
    #barcode position not in any genes
    if not gene_list:
        return ''
    #barcode position is in one gene
    if len(gene_list) == 1:
        return gene_list[0]
    #barcode position is in multiple genes (should be rare)
    else:
        return ';'.join(gene_list)

barcode_to_gene_dict = {}

for scaffold in sorted(gff['scaffold'].unique()):
    
    scaffold_gff = gff[gff['scaffold'] == scaffold]
    scaffold_barcodes = df[df['scaffold'] == scaffold]
    
    # Count unique genes
    unique_genes_count = scaffold_gff['ID'].nunique()
    
    print(f'Mapping {scaffold_barcodes.shape[0]} barcodes on {scaffold} to {unique_genes_count} genes on {scaffold}')

    # Turn the start and stop of each gene into an interval
    v = scaffold_gff.loc[:, 'start':'stop'].apply(tuple, axis=1).tolist()
    idx = pd.IntervalIndex.from_tuples(v, closed='both') 
    scaffold_gff = scaffold_gff.set_index(idx)
    
    # For each barcode, check if it's in each start/stop interval
    barcode_to_gene_dict.update(scaffold_barcodes['pos'].apply(get_gene_by_bc_position).to_dict())
    
df['gene'] = df.index.map(barcode_to_gene_dict)


Mapping 4186 barcodes on chr1 to 2712 genes on chr1
Mapping 2234 barcodes on chr2 to 1430 genes on chr2
Mapping 1732 barcodes on chr3 to 1210 genes on chr3
Mapping 1266 barcodes on chr4 to 1070 genes on chr4
Mapping 1040 barcodes on chr5 to 985 genes on chr5
Mapping 1016 barcodes on chr6 to 793 genes on chr6
Mapping 1890 barcodes on chr7 to 910 genes on chr7


In [24]:
#check that a 'gene' column has been added to df

df.head()

,rcbarcode,nTot,n,scaffold,strand,pos,type,nMainLocation,nInsert,All genomic mappings,gene
barcode,,,,,,,,,,,
TTCATTCAGGATCTTTGCTC,GAGCAAAGATCCTGAATGAA,601,601,chr1,+,8764372,Single,563,14,"[('chr1:8764372:+', 563), ('chr5:3325007:-', 3), ('chr7:2406660:-', 2), ('chr6:477092:-', 1), ('chr7:1928216:+', 1), ('chr4:2631691:-', 1), ('chr3:313312:+', 1), ('chr6:733284:-', 1), ('chr1:4277183:-', 1), ('chr1:1979828:+', 1), ('chr2:991296:-', 1), ('chr3:838055:-', 1), ('chr3:3091523:+', 1), ('chr1:7763084:+', 1), ('chr6:3244298:+', 1), ('chr1:6914244:-', 1), ('chr3:2416675:+', 1), ('chr1:9602398:+', 1), ('chr1:10585823:+', 1), ('chr1:8696906:+', 1), ('chr4:2000475:-', 1), ('chr7:1733638:-', 1)]",proteinId=2298894
ACGTATTCGCAGTCCTCATA,TATGAGGACTGCGAATACGT,52,52,chr1,-,526973,Single,50,0,"[('chr1:526973:-', 50), ('chr1:972680:+', 1), ('chr6:932721:-', 1)]",
GCGGCGGCAGCGTTCCTCGC,GCGAGGAACGCTGCCGCCGC,498,498,chr1,-,2235514,Single,450,14,"[('chr1:2235514:-', 450), ('chr7:2406660:-', 2), ('chr7:159854:-', 2), ('chr2:2382780:-', 1), ('chr3:2906544:-', 1), ('chr2:2797042:-', 1), ('chr3:1386264:-', 1), ('chr1:9117601:+', 1), ('chr1:2235513:-', 1), ('chr2:1023360:+', 1), ('chr5:3144776:-', 1), ('chr5:3773802:+', 1), ('chr5:3199747:-', 1), ('chr7:3111985:+', 1), ('chr1:4277183:-', 1), ('chr1:6419382:-', 1), ('chr3:1018484:+', 1), ('chr2:1802697:+', 1), ('chr1:8693211:+', 1), ('chr3:409126:+', 1), ('chr1:9400362:+', 1), ('chr5:3325007:-', 1), ('chr1:3432198:+', 1), ('chr2:2283320:+', 1), ('chr4:2420774:+', 1), ('chr6:3402852:+', 1), ('chr3:2949454:+', 1), ('chr7:2350573:+', 1), ('chr4:2678882:+', 1), ('chr2:4643869:-', 1), ('chr7:3369397:+', 1), ('chr7:3502691:-', 1), ('chr1:9554108:-', 1)]",
TTCGCGTGGGTGAATTAACT,AGTTAATTCACCCACGCGAA,1806,1806,chr5,+,2435148,Single,1660,43,"[('chr5:2435148:+', 1660), ('chr7:2406660:-', 10), ('chr3:2141768:+', 3), ('chr5:3325007:-', 3), ('chr1:9415816:+', 3), ('chr5:2435149:+', 3), ('chr5:2435205:-', 2), ('chr1:9554108:-', 2), ('chr3:2949780:+', 2), ('chr6:201290:+', 2), ('chr1:6914244:-', 2), ('chr5:2633031:-', 2), ('chr1:7572780:-', 1), ('chr2:1803458:-', 1), ('chr5:2574087:-', 1), ('chr7:2035049:-', 1), ('chr7:2775803:+', 1), ('chr3:2410718:-', 1), ('chr3:2618659:-', 1), ('chr1:6943496:-', 1), ('chr2:4994449:+', 1), ('chr2:4661375:+', 1), ('chr7:1074090:+', 1), ('chr7:1049234:+', 1), ('chr6:3125324:-', 1), ('chr2:4512077:+', 1), ('chr2:1751841:-', 1), ('chr1:6903684:-', 1), ('chr1:391550:-', 1), ('chr2:3965159:+', 1), ('chr7:1885715:-', 1), ('chr2:2382780:-', 1), ('chr5:464496:-', 1), ('chr2:3730704:+', 1), ('chr1:6024539:+', 1), ('chr1:6347373:+', 1), ('chr1:6012377:-', 1), ('chr1:9969230:-', 1), ('chr3:142030:+', 1), ('chr1:1505216:+', 1), ('chr2:5223225:-', 1), ('chr2:5364748:+', 1), ('chr1:7624221:-', 1), ('chr5:2435151:+', 1), ('chr6:1178030:-', 1), ('chr2:4314401:+', 1), ('chr6:1653165:-', 1), ('chr1:3843797:+', 1), ('chr5:2435190:-', 1), ('chr3:2931743:-', 1), ('chr2:2044521:+', 1), ('chr4:2071158:-', 1), ('chr2:1751846:+', 1), ('chr2:4994013:-', 1), ('chr1:6383044:-', 1), ('chr1:2396701:+', 1), ('chr3:3753786:-', 1), ('chr7:2321031:+', 1), ('chr4:127963:+', 1), ('chr4:2045081:+', 1), ('chr2:1589959:-', 1), ('chr2:3539420:+', 1), ('chr7:2206210:+', 1), ('chr6:2450270:-', 1), ('chr1:7383411:-', 1), ('chr7:2484936:+', 1), ('chr1:1517359:-', 1), ('chr7:3111985:+', 1), ('chr7:2626073:-', 1), ('chr1:821751:-', 1), ('chr7:601923:+', 1), ('chr3:3135700:-', 1), ('chr5:3772661:+', 1), ('chr5:3773803:+', 1), ('chr1:9316561:-', 1), ('chr1:3703136:-', 1), ('chr3:4590094:-', 1), ('chr6:2851498:+', 1), ('chr2:1807179:-', 1), ('chr7:1926954:-', 1), ('chr2:4267615:+', 1)]",
TTCGTTTAGTTGTCTAAGTA,TACTTAGACAACTAAACGAA,401,401,chr3,-,1481765,Concat,318,50,"[('chr3:1481765:-', 318), ('chr7:2406660:-', 2), ('chr1:2541261:+', 2), ('chr1:6517427:-', 1), ('chr1:8477319:-', 1), ('chr7:2592385:-', 1), ('chr5:2283227:-', 1), ('chr3:3582433:+', 1), ('chr2:3041586:-', 1), ('chr1:9415816:+', 1), ('chr1:9120459:+', 1), ('chr2:1702333:-'

### This line is not necessary in the running. Continue to the next line to save the dataframe

In [ ]:
print('{} barcodes not in coding sequences'.format(df[df['gene']==''].shape[0]))
print()

print('{} barcodes in coding sequences'.format(df[df['gene']!=''].shape[0]))
print('{} barcodes in Kl coding sequences'.format(df[df['gene'].str.startswith('kl',na=False)].shape[0]))
print('{} barcodes in Km coding sequences'.format(df[df['gene'].str.startswith('km',na=False)].shape[0]))

In [25]:
df.to_csv(annotated_poolfile_path, sep='\t')

In [ ]:
gff[gff['ID']=='kmYHR034C']